In [67]:
# Imports
from src.vector_utils import *
from src.distances import *
from src.print import *
from typing import Callable
import pandas as pd
import time

In [68]:
# Dictionnary test and inputs
tests: dict = {
	"Manhattan":				{"function":distance_manhattan},
	"Euclidean":				{"function":distance_euclidean},
	"Tchebyshev":				{"function":distance_tchebyshev},
	"Minkowski":				{"function":distance_minkowski},
	"Histogram Intersection":	{"function":distance_histogram_intersection},
	"Swapped Histogram":		{"function":distance_histogram_intersection},
	"Khi2":						{"function":distance_khi2},
}

In [69]:
## Prepare inputs
# Two known vectors for other tests
v1: np.ndarray = np.array([2.7, 4.3,  0.2,   9,   -4])
v2: np.ndarray = np.array([7.6, 5.8, -3.2, 9.7, 12.3])

# Generate two random vectors
v3: np.ndarray = np.round(random_vector(10), 4)
v4: np.ndarray = np.round(random_vector(10), 4)

# Similar vectors
v5: np.ndarray = np.array([1, 2, 3, 4, 5])
v6: np.ndarray = np.array([1, 2, 3, 4, 5])

# Prepare input tuples
inputs: list[tuple] = [
	("v1/v2 (vecteurs du cours)", v1, v2),
	("v3/v4 (vecteurs aléatoires)", v3, v4),
	("v5/v6 (vecteurs similaires)", v5, v6),
]

In [70]:
# Test all the inputs
for input_type, x, y in inputs:
	for method, test in tests.items():
		f: Callable = test["function"]
		
		# Measure the time
		start: int = time.perf_counter_ns()
		distance: float = f(x,y) if not "Swap" in method else f(y,x)
		end: int = time.perf_counter_ns()
		duration: int = end - start

		# Store the result
		test[input_type] = {"distance":distance, "duration":duration}

In [71]:
# Util function to print the results
def get_dataframe(tests: dict, input_type: str) -> pd.DataFrame:
	""" Get the dataframe for the given input type\n
	Args:
		tests		(dict):	Dictionnary containing every tests
		input_type	(str):	Input type (text for which inputs are used)
	Returns:
		(pd.DataFrame):	Dataframe containing the results
	"""
	# Prepare the dataframe
	df: pd.DataFrame = pd.DataFrame()
	for name in tests.keys():
		df[name] = 0
	
	# Add the distances and duration line
	for name in tests.keys():
		df.loc["Distance", name] = round(tests[name][input_type]["distance"], 4)
		df.loc["Time (ns)", name] = round(tests[name][input_type]["duration"])

	# Return the dataframe
	return df

In [72]:
# Show first inputs (made in class)
input_type: str = inputs[0][0]
x = inputs[0][1]
y = inputs[0][2]
info(f"{input_type}\n x = {x}\n y = {y}")
get_dataframe(tests, input_type)

[INFO  13:53:21] v1/v2 (vecteurs du cours)
 x = [ 2.7  4.3  0.2  9.  -4. ]
 y = [ 7.6  5.8 -3.2  9.7 12.3]


,Manhattan,Euclidean,Tchebyshev,Minkowski,Histogram Intersection,Swapped Histogram,Khi2
Distance,26.8,17.4356,16.3,19.3847,0.2733,0.7213,5.3909
Time (ns),175100.0,69000.0000,44900.0,73300.0000,59100.0000,48200.0000,97400.0000


In [73]:
# Show second inputs (random vectors)
input_type: str = inputs[1][0]
x = inputs[1][1]
y = inputs[1][2]
info(f"{input_type}\n x = {x}\n y = {y}")
get_dataframe(tests, input_type)

[INFO  13:53:22] v3/v4 (vecteurs aléatoires)
 x = [0.794  0.9838 0.7    0.4261 0.1267 0.864  0.6406 0.5133 0.508  0.6377]
 y = [0.0962 0.0218 0.6637 0.7161 0.8883 0.7136 0.1804 0.7669 0.8094 0.9936]


,Manhattan,Euclidean,Tchebyshev,Minkowski,Histogram Intersection,Swapped Histogram,Khi2
Distance,4.2692,1.6106,0.962,2.1852,0.6645,0.6276,2.6203
Time (ns),37200.0000,40800.0000,35300.000,62600.0000,53000.0000,48200.0000,85700.0000


In [74]:
# Show third inputs (similar vectors)
input_type: str = inputs[2][0]
x = inputs[2][1]
y = inputs[2][2]
info(f"{input_type}\n x = {x}\n y = {y}")
get_dataframe(tests, input_type)

[INFO  13:53:24] v5/v6 (vecteurs similaires)
 x = [1 2 3 4 5]
 y = [1 2 3 4 5]


,Manhattan,Euclidean,Tchebyshev,Minkowski,Histogram Intersection,Swapped Histogram,Khi2
Distance,0.0,0.0,0.0,0.0,1.0,1.0,0.0
Time (ns),60600.0,47800.0,36200.0,56000.0,49000.0,46500.0,75600.0


In [79]:
# Sort algorithms in terms of speed
df = pd.DataFrame()
time_str: str = "Total Time (ns)"
df[time_str] = 0
for method in tests.keys():
	df.loc[method, time_str] = sum(tests[method][input_type[0]]["duration"] for input_type in inputs)
df = df.sort_values(by=time_str)
df[time_str] = df[time_str].astype(int)
info("Total time for each method")
df

[INFO  14:00:29] Total time for each method


,Total Time (ns)
Tchebyshev,116400
Swapped Histogram,142900
Euclidean,157600
Histogram Intersection,161100
Minkowski,191900
Khi2,258700
Manhattan,272900
